# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from config import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Study data files
cities_path = "C:/Users/lilia/Desktop/git/python-api-challenge/Instructions/output_data/City_Data_Final.csv"

# Read the cities data 
cities_data = pd.read_csv(cities_path)



### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
# Store latitude and longitude in locations
locations = cities_data[['Lat','Lng']]

# Convert to float
humidity = cities_data['Humidity'].astype(float)

#Getting the maximum humidity
max_hum = cities_data['Humidity'].max()
max_hum

100

In [24]:
# Plot Heatmap
fig = gmaps.figure(zoom_level=2, center=(0,0))



# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)



# Add layer
fig.add_layer(heat_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_cities = cities_data.loc[(cities_data['Max Temp']<= 80) & (cities_data['Max Temp']> 70) & (cities_data['Wind Speed']<= 10) & (cities_data['Cloudiness']== 0)].dropna()

ideal_cities.head()

City_ID         City    Lat     Lng  Max Temp  Humidity  Cloudiness  \
27        27     algemesi  39.18   -0.43     71.01        87           0   
71        71     udachnyy  66.42  112.40     75.20        29           0   
110      110       toledo  39.83   -4.00     79.00        47           0   
124      124  georgiyevka  43.04   74.71     71.60        43           0   
191      191       rizhao  35.43  119.46     78.31        59           0   

     Wind Speed Country        Date  
27         1.01      ES  1593052654  
71         6.71      RU  1593052657  
110        6.93      ES  1593052517  
124        6.71      KZ  1593052663  
191        4.74      CN  1593052669

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_cities

#Adding a column to the DataFrame
hotel_df['Hotel Name'] = ''
hotel_df

City_ID                 City    Lat     Lng  Max Temp  Humidity  \
27        27             algemesi  39.18   -0.43     71.01        87   
71        71             udachnyy  66.42  112.40     75.20        29   
110      110               toledo  39.83   -4.00     79.00        47   
124      124          georgiyevka  43.04   74.71     71.60        43   
191      191               rizhao  35.43  119.46     78.31        59   
195      195  sao raimundo nonato  -9.02  -42.70     71.69        69   
268      268                altay  47.87   88.12     73.26        36   
287      287       elesbao veloso  -6.20  -42.14     71.62        57   
294      294          yerbogachen  61.28  108.01     72.52        46   
319      319               aykhal  66.00  111.50     75.20        29   
320      320          mitsamiouli -11.38   43.28     78.04        75   
372      372                anzio  41.49   12.62     71.01        87   
383      383            morondava -20.28   44.28     70.05        90   
487      487                ozark  37.02  -93.21     73.99        77   
522      522          deputatskiy  69.30  139.90     78.93        28   
533      533               sivaki  52.63  126.75     74.71        39   
561      561        mokhsogollokh  61.47  128.92     76.55        38   

     Cloudiness  Wind Speed Country        Date Hotel Name  
27            0        1.01      ES  1593052654             
71            0        6.71      RU  1593052657             
110           0        6.93      ES  1593052517             
124           0        6.71      KZ  1593052663             
191           0        4.74      CN  1593052669             
195           0        4.81      BR  1593052669             
268           0        4.50      CN  1593052676             
287           0        3.71      BR  1593052680             
294           0        4.32      RU  1593052680             
319           0        6.71      RU  1593052385             
320           0        9.78      KM  1593052683             
372           0        1.01      IT  1593052688             
383           0        8.97      MG  1593052688             
487           0        0.43      US  1593052706             
522           0        8.55      RU  1593052708             
533           0        3.83      RU  1593052710             
561           0        7.23      RU  1593052711

In [7]:
# geocoordinates
target_search = "nearby hotel"
target_radius = 8000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Loop through the hotel_df and run a nearby search for each city
for index, row in hotel_df.iterrows():
    #get name of the city and the coordinates
    city = row['City']
    Lat = row['Lat']
    Lng = row['Lng']

    # update location key value
    params['location'] = f"{Lat},{Lng}"

    # make request
    nearby_hotel = requests.get(base_url, params=params)
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    nearby_hotel = requests.get(base_url, params=params).json()
    
    # extract results
    results = nearby_hotel['results']
    
    #Save to DataFrame
    try:
        print(f"Closest {target_type} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    #Add a time sleep to avoid SSL Error
    time.sleep(1)    
    
    print("------------")

print("SEARCH COMPLETE------------")    


Retrieving Results for Index 27: algemesi.
Closest hotel is Hotel Avenida Plaza.
------------
Retrieving Results for Index 71: udachnyy.
Closest hotel is Set' Apartamentov "Vizit".
------------
Retrieving Results for Index 110: toledo.
Closest hotel is Entre Dos Aguas Hotel Boutique.
------------
Retrieving Results for Index 124: georgiyevka.
Missing field/result... skipping.
------------
Retrieving Results for Index 191: rizhao.
Closest hotel is Novotel Rizhao Suning.
------------
Retrieving Results for Index 195: sao raimundo nonato.
Closest hotel is Real Hotel.
------------
Retrieving Results for Index 268: altay.
Closest hotel is SILVERWATER HOTEL.
------------
Retrieving Results for Index 287: elesbao veloso.
Missing field/result... skipping.
------------
Retrieving Results for Index 294: yerbogachen.
Missing field/result... skipping.
------------
Retrieving Results for Index 319: aykhal.
Closest hotel is Hotel Astoria.
------------
Retrieving Results for Index 320: mitsamiouli.
C

In [8]:
hotel_df


City_ID                 City    Lat     Lng  Max Temp  Humidity  \
27        27             algemesi  39.18   -0.43     71.01        87   
71        71             udachnyy  66.42  112.40     75.20        29   
110      110               toledo  39.83   -4.00     79.00        47   
124      124          georgiyevka  43.04   74.71     71.60        43   
191      191               rizhao  35.43  119.46     78.31        59   
195      195  sao raimundo nonato  -9.02  -42.70     71.69        69   
268      268                altay  47.87   88.12     73.26        36   
287      287       elesbao veloso  -6.20  -42.14     71.62        57   
294      294          yerbogachen  61.28  108.01     72.52        46   
319      319               aykhal  66.00  111.50     75.20        29   
320      320          mitsamiouli -11.38   43.28     78.04        75   
372      372                anzio  41.49   12.62     71.01        87   
383      383            morondava -20.28   44.28     70.05        90   
487      487                ozark  37.02  -93.21     73.99        77   
522      522          deputatskiy  69.30  139.90     78.93        28   
533      533               sivaki  52.63  126.75     74.71        39   
561      561        mokhsogollokh  61.47  128.92     76.55        38   

     Cloudiness  Wind Speed Country        Date  \
27            0        1.01      ES  1593052654   
71            0        6.71      RU  1593052657   
110           0        6.93      ES  1593052517   
124           0        6.71      KZ  1593052663   
191           0        4.74      CN  1593052669   
195           0        4.81      BR  1593052669   
268           0        4.50      CN  1593052676   
287           0        3.71      BR  1593052680   
294           0        4.32      RU  1593052680   
319           0        6.71      RU  1593052385   
320           0        9.78      KM  1593052683   
372           0        1.01      IT  1593052688   
383           0        8.97      MG  1593052688   
487           0        0.43      US  1593052706   
522           0        8.55      RU  1593052708   
533           0        3.83      RU  1593052710   
561           0        7.23      RU  1593052711   

                         Hotel Name  
27              Hotel Avenida Plaza  
71        Set' Apartamentov "Vizit"  
110  Entre Dos Aguas Hotel Boutique  
124                                  
191           Novotel Rizhao Suning  
195                      Real Hotel  
268               SILVERWATER HOTEL  
287                                  
294                                  
319                   Hotel Astoria  
320                      Moina Lisa  
372                 Hotel l'Approdo  
383             Kimony Resort Hotel  
487                     Quality Inn  
522                                  
533                                  
561

In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Hotel Avenida Plaza</dd>\n<dt>City</dt><dd>algemesi</dd>\n<dt>Country</dt><dd>ES</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Set\' Apartamentov "Vizit"</dd>\n<dt>City</dt><dd>udachnyy</dd>\n<dt>Country</dt><dd>RU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Entre Dos Aguas Hotel Boutique</dd>\n<dt>City</dt><dd>toledo</dd>\n<dt>Country</dt><dd>ES</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>georgiyevka</dd>\n<dt>Country</dt><dd>KZ</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Novotel Rizhao Suning</dd>\n<dt>City</dt><dd>rizhao</dd>\n<dt>Country</dt><dd>CN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Real Hotel</dd>\n<dt>City</dt><dd>sao raimundo nonato</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>SILVERWATER HOTEL</dd>\n<dt>City</dt><dd>altay</dd>\n<dt>Country</dt><dd>CN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>elesbao veloso</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><

In [27]:
# Add marker layer ontop of heat map


markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))